# Data Loading and Cleaning

In [15]:
# imports
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [16]:
# Loading csv with preprocessed data
df = pd.read_csv('/Users/mep/Desktop/Preprocessed_data.csv')

# Cleaning the data
# Removing zero ratings
df = df[df['rating'] != 0]

In [18]:
# Removing image_url columns because we will not use it.
df = df.drop(['img_s'], axis=1)
df = df.drop(['img_m'], axis=1)
df = df.drop(['img_l'], axis=1)

In [19]:
print (df)

         Unnamed: 0  user_id                   location      age        isbn  \
1                 1        8   timmins, ontario, canada  34.7439  0002005018   
3                 3    11676              n/a, n/a, n/a  34.7439  0002005018   
5                 5    67544   toronto, ontario, canada  30.0000  0002005018   
8                 8   116866                  ottawa, ,  34.7439  0002005018   
9                 9   123629  kingston, ontario, canada  34.7439  0002005018   
...             ...      ...                        ...      ...         ...   
1031169     1031169   278851         dallas, texas, usa  33.0000  067161746X   
1031171     1031171   278851         dallas, texas, usa  33.0000  0767907566   
1031172     1031172   278851         dallas, texas, usa  33.0000  0884159221   
1031173     1031173   278851         dallas, texas, usa  33.0000  0912333022   
1031174     1031174   278851         dallas, texas, usa  33.0000  1569661057   

         rating                        

In [14]:
df.to_csv('Preprocessed_data_cleaned.csv', index=False)

In [20]:
# From now on we can work with df or load from the beggining 
# the Preprocessed_data_cleaned.csv file

-------------------------------------------------------------------
# Demographic Filtering

------------------------------------------------------------------------
# Content Filtering

In [1]:
print('content')

content
